In [1]:
import pandas as pd

In [2]:
def create_train2(X):
    #set month in each example
    X['month'] = X['date'].dt.month
    #set week in month. We want at most 4
    X['week'] = X.date.apply(lambda d: int(min((d.day-1)//7 + 1, 4)))
    #every existing combination of userID and itemID
    user_items = X[['userID', 'itemID']].groupby(['userID', 'itemID']).all().reset_index()
    user_items['week'] = 5
    months = pd.DataFrame({
        'month':X.month.unique()
    })
    #every combination of existing userIDxitemID pairs with every month found in the data
    user_items = user_items.merge(months, how='cross')
    user_item_month_week = X[['month', 'userID', 'itemID', 'week']]
    user_item_month_week = pd.concat([user_item_month_week, user_items])
    '''find for every userxitemxmonth combination the min week. If the combination doesn't exist in the data, 
    the week will be 5 since we added all those combinations in the step before
    Afterwards we set week to 0 where week=5, since these combinations weren't present in the data
    '''
    res = user_item_month_week.groupby(['month', 'userID', 'itemID']).min().reset_index()
    res['week'] = res.week.apply(lambda w: w if w < 5 else 0)
    return res

In [3]:
orders_before_dec = pd.read_csv('../../Data/orders_before_dec.csv', delimiter='|', parse_dates=[0])

In [4]:
train1 = create_train2(orders_before_dec)

In [5]:
train1.to_csv('../../Data/train_before_dec_v1.csv', sep='|', index=False)

In [6]:
orders_before_jan = pd.read_csv('../../Data/orders_before_jan.csv', delimiter='|', parse_dates=[0])

In [7]:
train2 = create_train2(orders_before_jan)

In [8]:
train2.to_csv('../../Data/train_before_jan_v1.csv', sep='|', index=False)

In [9]:
test1 = pd.read_csv('../../Data/orders.csv', delimiter='|', parse_dates=[0])

In [10]:
test1 = create_train2(test1)

In [11]:
test2 = pd.read_csv('../../Data/submission_dec.csv', delimiter='|')

In [12]:
test2 = test2.drop(columns=['prediction'])
test2['month'] = 12

In [13]:
test_dec = test2.merge(test1, how='left')

In [14]:
test_dec.head()

userID  itemID  month  week
0       0   20664     12     2
1       0   28231     12     2
2      13    2690     12     4
3      15    1299     12     4
4      15   20968     12     4

In [15]:
test_dec['prediction'] = test_dec['week']

In [16]:
test_dec = test_dec.drop(columns=['month', 'week'])

In [17]:
test_dec.to_csv('../Data/test_dec.csv', sep='|', index=False)

In [18]:
test3 = pd.read_csv('../../Data/submission_jan.csv', delimiter='|')

In [19]:
test3['month'] = 1
test3 = test3.drop(columns=['prediction'])

In [20]:
test_jan = test3.merge(test1, how='left')

In [21]:
test_jan['prediction'] = test_jan['week']
test_jan = test_jan.drop(columns=['month', 'week'])

In [22]:
test_jan.to_csv('../../Data/test_jan.csv', sep='|', index=False)